＃ 使用ｐｙｔｏｒｃｈ训练Ｒｅｓｎｅｔ５０

In [6]:
import torch
import torchvision.models as models
import torch.nn as nn
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
# if choose pretrained, the model will be downloaded at ~/.torch 
resnet50 = models.resnet50(pretrained=True)
# print(resnet50) # print network structure

# change classification into 110. Original classification is 1000 from ISLVRC 2012.
fc_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(fc_features,110) 
print(resnet50) # print new network structure

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [8]:
from torch import optim

loss_fn = nn.CrossEntropyLoss() # 定义损失函数
optimizer = optim.SGD(params=resnet50.parameters(), lr=0.01, momentum=0.5) # 定义优化器

In [53]:
# 数据预处理
import os
from PIL import Image
from torch.utils import data
from torchvision import transforms
import csv

class MyData(data.Dataset):
    def __init__(self, root, trans=None, train=True):
        self.root = root
        self.trans = trans
        self.train = train
        
        self.read_csv_file()
        
        if self.trans is None:
            normalize = transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
            if not train: # test
                self.trans = transforms.Compose([
                    transforms.Scale(224),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    normalize
                    ]) 
            else: # train
                self.trans = transforms.Compose([
                    transforms.Pad(50, fill=0),
                    transforms.RandomCrop(256),
                    transforms.RandomSizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    normalize])
                
    def read_csv_file(self):
        self.images_path = []
        self.images_labels = []

        csv_file = os.path.join(self.root, "dev.csv")
        with open(csv_file, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                filepath = os.path.join(self.root, row['filename'])
                self.images_path.append(filepath)
                self.images_labels.append(row['trueLabel'])
                
    def __getitem__(self,index):
        label = self.images_labels[index]
        image = Image.open(self.images_path[index])
        image = self.trans(image)
        return image,int(label)
    
    def __len__(self):
        return len(self.images_path)

In [54]:
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.datasets import *
import torch.utils.data as Data
import numpy as np
import random

all_data = MyData(root='./dev_data')
dataset_size = len(all_data)
print('dataset length:',dataset_size)
indices = list(range(dataset_size))


dataset length: 110


In [55]:
# 通过ｉｎｄｅｘ生成ｔｒａｉｎ/val数据集
def train_val_division():
    random.shuffle(indices)
    print('new indices:',indices)
    validation_split = 0.2
    split = int(np.floor(validation_split*dataset_size)) 
    train_indices, val_indices = indices[split:], indices[:split]
    print('devide into:',train_indices,val_indices) # 重新划分的数据集
    train_sampler = Data.sampler.SubsetRandomSampler(train_indices)
    valid_sampler = Data.sampler.SubsetRandomSampler(val_indices)
    
    # sampler和shuffle不能重用． batch_size由网络结构决定
    train_loader = Data.DataLoader(dataset=all_data, batch_size=64, sampler=train_sampler)
    validation_loader = Data.DataLoader(dataset=all_data, batch_size=64, sampler=valid_sampler)
    return train_loader, validation_loader

In [56]:
import torch.nn.functional as F
# 每个ｅｐｏｃｈ进行一次训练和验证
def train_val_per_epoch(epoch):
    train_loader,val_loader = train_val_division()
    resnet50.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = resnet50(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        print('epoch',epoch,': batch',batch_idx,': loss',loss.item())
        
    resnet50.eval()
    score = 0.0
    for step, (data, target) in enumerate(val_loader):
        print(step)
        data, target = Variable(data), Variable(target)
        score = resnet50(data)
        # score 平均或损失矩阵
        _, prediction = torch.max(score.data, dim=1)
        print('Prediction number: ', prediction[0])


In [57]:
# 虚拟机空间不够了．训练部分的代码没有问题，谁电脑厉害谁跑...
epoch = 2
for e in range(epoch):
    train_val_per_epoch(e)
    # 每个ｅｐｏｃｈ保存一次模型
    torch.save(resnet50, './save/resnet50-epoch'+str(e)+'.pkl')
    torch.save(resnet50.state_dict(),'./save/res50_params.pkl')

new indices: [12, 96, 80, 59, 44, 47, 38, 78, 39, 87, 16, 5, 93, 45, 90, 23, 24, 14, 4, 34, 1, 69, 49, 104, 10, 81, 53, 42, 54, 88, 98, 0, 46, 63, 27, 95, 6, 29, 9, 72, 105, 57, 55, 68, 100, 85, 67, 108, 15, 3, 17, 103, 89, 71, 109, 35, 52, 18, 43, 37, 107, 106, 73, 2, 82, 76, 51, 8, 30, 75, 28, 11, 84, 56, 65, 60, 25, 40, 26, 101, 94, 19, 83, 64, 77, 61, 58, 20, 48, 13, 102, 66, 86, 62, 21, 33, 97, 7, 41, 92, 74, 91, 32, 22, 31, 99, 79, 50, 36, 70]
devide into: [49, 104, 10, 81, 53, 42, 54, 88, 98, 0, 46, 63, 27, 95, 6, 29, 9, 72, 105, 57, 55, 68, 100, 85, 67, 108, 15, 3, 17, 103, 89, 71, 109, 35, 52, 18, 43, 37, 107, 106, 73, 2, 82, 76, 51, 8, 30, 75, 28, 11, 84, 56, 65, 60, 25, 40, 26, 101, 94, 19, 83, 64, 77, 61, 58, 20, 48, 13, 102, 66, 86, 62, 21, 33, 97, 7, 41, 92, 74, 91, 32, 22, 31, 99, 79, 50, 36, 70] [12, 96, 80, 59, 44, 47, 38, 78, 39, 87, 16, 5, 93, 45, 90, 23, 24, 14, 4, 34, 1, 69]


RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at /pytorch/aten/src/TH/THGeneral.cpp:201

In [58]:
model = torch.load('./save/resnet50-epoch1.pkl')

In [66]:
import torchvision.transforms.functional as tv_F
import torch.nn.functional as F

im2 = Image.open('./im2.png')
im3 = Image.open('./im3.png')
im4 = Image.open('./im4.png')
normalize = transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])

def test(img):
    model.eval()
    img = tv_F.to_tensor(tv_F.resize(img, (224, 224)))
    img = tv_F.normalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    img_input = Variable(torch.unsqueeze(img, 0))

    output = resnet50(img_input)
    _, prediction = torch.max(output.data, dim=1)
    print('Prediction number: ', prediction[0])
#     for batch_idx, data in enumerate(val_loader):
#         data = Variable(data)
#         output = model(data)
#     print(output)

# test(im1)
test(im2)
test(im3)
test(im4)

Prediction number:  tensor(1)
Prediction number:  tensor(2)
Prediction number:  tensor(7)
